## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,26.60,100,75,11.50,broken clouds
1,1,Ulladulla,AU,-35.3500,150.4667,62.10,71,100,5.19,overcast clouds
2,2,Arenapolis,BR,-14.4503,-56.8461,79.02,75,90,1.97,overcast clouds
3,3,Mahebourg,MU,-20.4081,57.7000,71.26,94,40,3.44,scattered clouds
4,4,Severo-Kurilsk,RU,50.6789,156.1250,48.11,69,21,17.49,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Arenapolis,BR,-14.4503,-56.8461,79.02,75,90,1.97,overcast clouds
3,3,Mahebourg,MU,-20.4081,57.7000,71.26,94,40,3.44,scattered clouds
6,6,Abu Zabad,SD,12.3500,29.2500,73.31,88,100,10.20,moderate rain
7,7,Lorengau,PG,-2.0226,147.2712,85.69,72,16,8.05,few clouds
8,8,Klaeng,TH,12.7797,101.6483,83.16,100,100,10.36,overcast clouds
13,13,Gamba,GA,-2.6500,10.0000,76.53,90,100,6.40,light rain
16,16,Hilo,US,19.7297,-155.0900,83.28,82,90,11.50,light rain
17,17,Jamestown,US,42.0970,-79.2353,71.19,82,90,5.75,overcast clouds
21,21,Victoria,HK,22.2855,114.1577,80.26,94,100,1.01,moderate rain
22,22,Umm Lajj,SA,25.0213,37.2685,85.12,50,0,10.49,clear sky


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                335
City                   335
Country                334
Lat                    335
Lng                    335
Max Temp               335
Humidity               335
Cloudiness             335
Wind Speed             335
Current Description    335
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arenapolis,BR,79.02,overcast clouds,-14.4503,-56.8461,
3,Mahebourg,MU,71.26,scattered clouds,-20.4081,57.7000,
6,Abu Zabad,SD,73.31,moderate rain,12.3500,29.2500,
7,Lorengau,PG,85.69,few clouds,-2.0226,147.2712,
8,Klaeng,TH,83.16,overcast clouds,12.7797,101.6483,
13,Gamba,GA,76.53,light rain,-2.6500,10.0000,
16,Hilo,US,83.28,light rain,19.7297,-155.0900,
17,Jamestown,US,71.19,overcast clouds,42.0970,-79.2353,
21,Victoria,HK,80.26,moderate rain,22.2855,114.1577,
22,Umm Lajj,SA,85.12,clear sky,25.0213,37.2685,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arenapolis,BR,79.02,overcast clouds,-14.4503,-56.8461,Mato Grosso Hotel
3,Mahebourg,MU,71.26,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Abu Zabad,SD,73.31,moderate rain,12.3500,29.2500,حامد محمد نور
7,Lorengau,PG,85.69,few clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
8,Klaeng,TH,83.16,overcast clouds,12.7797,101.6483,สวัสดีการ์เดนท์รีสอร์ท
16,Hilo,US,83.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Jamestown,US,71.19,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
21,Victoria,HK,80.26,moderate rain,22.2855,114.1577,Mini Hotel Central
22,Umm Lajj,SA,85.12,clear sky,25.0213,37.2685,Moon light Furnished Units
23,Ponta Do Sol,PT,69.71,clear sky,32.6667,-17.1000,Hotel do Campo


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))